## Failed Experiment of generating journals data using llama 7b
the results were too much robotic  

## Hugging Face login

In [ ]:
from huggingface_hub import login
login()


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install model dependencies

In [ ]:
!pip install -q transformers accelerate bitsandbytes sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True,     # Fits inside A100 nicely
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## Journal generation helper

In [ ]:
def generate_journal_entry(
    primary_emotion=None,
    secondary_emotions=None,
    length="medium",  # "short", "medium", "long"
):
    length_map = {
        "short": "2–4 sentences",
        "medium": "5–8 sentences",
        "long": "10–15 sentences",
    }

    emotional_clause = ""
    if primary_emotion:
        emotional_clause += f"The entry should *implicitly* reflect the primary emotion: {primary_emotion}. "
    if secondary_emotions:
        emotional_clause += f"It may contain subtle traces of: {', '.join(secondary_emotions)}. "

    user_content = f"""
Write a realistic first-person journaling entry.

Rules:
- Do NOT use words like "sad", "angry", "anxious" explicitly.
- SHOW the emotions through thoughts, behavior, body sensations, and dialogue.
- Include some personal context (age, background, relationships, work, culture).
- Keep it natural, messy, and human.
- Do NOT explain the emotions, do NOT analyze the text, do NOT add headings or bullet points.

Length: {length_map[length]}.

{emotional_clause}

Output format:
- Return ONLY the journal entry text.
- No explanations, no commentary, no markdown, no lists.
"""

    messages = [
        {
            "role": "system",
            "content": (
                "You are a journaling simulator. "
                "When the user asks for a journal entry, you respond ONLY with the journal text itself. "
                "Never explain, never analyze, never add bullets or headings."
            ),
        },
        {"role": "user", "content": user_content},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=320,
        temperature=0.9,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    full = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Strip the prompt part and just keep the assistant answer
    # (everything after the last special token / prompt text)
    journal = full.split(user_content.strip())[-1].strip()

    return journal


## Quick sample

In [ ]:
entry = generate_journal_entry(
    primary_emotion="sadness",
    secondary_emotions=["shame"],
    length="medium",
)

print(entry)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


assistant

I woke up before dawn again to the sound of my mom's snores from downstairs. The smell of last night's leftover pho clings to my skin, an unpleasant reminder that I didn't eat anything decent after coming home from that disastrous date. My eyes stung as I rubbed the sleep from them, but it wasn't just tiredness - a sharp pang in my chest echoed each time I thought about Emily's smile, the way she laughed at things I said only to correct me mid-sentence. Mom called out asking if I was okay; I lied and said yeah, fine, what's up? Why did I even bother going on that date anyway? Just another attempt to fit into her mold of who I should be - successful, independent, happy. Sometimes I wonder if that's all there is to life: pretending until we're old enough to be miserable together.


## Batch 1: balanced primaries

In [ ]:
import pandas as pd

emotions = ["joy", "sadness", "anger", "fear", "caring", "disgust", "anticipation", "surprise", "neutral"]

rows = []

for e in emotions:
    for _ in range(80):  # 80 samples per emotion → 720 entries total
        entry = generate_journal_entry(
            primary_emotion=e,
            secondary_emotions=None,
            length="medium",
        )
        rows.append({"text": entry, "primary": e})

df = pd.DataFrame(rows)
df.head()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

,text,primary
0,assistant\n\nI woke up early today feeling lig...,joy
1,assistant\n\nToday's been one of those days wh...,joy
2,assistant\n\nI woke up before dawn to make mys...,joy
3,assistant\n\nI woke up early to the sound of c...,joy
4,assistant\n\nTired but still buzzing from last...,joy


## Batch 2: varied secondary emotions

In [ ]:
import random

# Same primary emotion list
emotions = ["joy", "sadness", "anger", "fear", "caring", "disgust", "anticipation", "surprise", "neutral"]

# Map primary → possible secondary emotions
secondary_map = {
    "joy": ["gratitude", "relief"],
    "sadness": ["shame", "regret"],
    "anger": ["disgust", "frustration"],
    "fear": ["anxiety", "worry"],
    "caring": ["concern", "tenderness"],
    "disgust": ["anger", "contempt"],
    "anticipation": ["hope", "nervousness"],
    "surprise": ["curiosity", "confusion"],
    "neutral": []
}

length_choices = ["short", "medium", "long"]

# Simple persona tags (metadata only for now)
personas = [
    "teenager",
    "university student",
    "young professional",
    "parent of two kids",
    "mid-career worker",
    "unemployed adult",
    "retired person",
]

rows2 = []

for e in emotions:
    for _ in range(80):  # another 80 per primary
        length = random.choice(length_choices)

        secs = secondary_map.get(e, [])
        if secs and random.random() < 0.6:
            secondary = [random.choice(secs)]
        else:
            secondary = None

        persona = random.choice(personas)

        entry = generate_journal_entry(
            primary_emotion=e,
            secondary_emotions=secondary,
            length=length,
        )

        rows2.append({
            "text": entry,
            "primary": e,
            "secondary": secondary[0] if secondary else None,
            "length_tag": length,
            "persona_tag": persona,
        })

df2 = pd.DataFrame(rows2)
df2.head(), len(df2)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

(                                                text primary  secondary  \
 0  assistant\n\nMarch 15th \nI'm sipping coffee o...     joy       None   
 1  assistant\n\nI woke up before dawn again, coul...     joy     relief   
 2  assistant\n\nI woke up early to the sound of b...     joy     relief   
 3  assistant\n\nI woke up early this morning to t...     joy  gratitude   
 4  assistant\n\nI woke up early today and stepped...     joy     relief   
 
   length_tag         persona_tag  
 0      short  parent of two kids  
 1       long  university student  
 2       long   mid-career worker  
 3     medium            teenager  
 4      short  young professional  ,
 720)

## Batch 3: length extremes

In [ ]:
# Batch 3 – push multi-emotion + length extremes

rows3 = []

for e in emotions:
    for _ in range(80):  # 80 per primary again
        # Bias toward very short or very long (fewer medium)
        length = random.choices(
            ["short", "medium", "long"],
            weights=[0.4, 0.2, 0.4],  # more short & long
            k=1
        )[0]

        # Stronger chance for secondary emotion
        secs = secondary_map.get(e, [])
        if secs and random.random() < 0.8:
            # maybe even 2 secondaries sometimes
            n_secs = 1 if random.random() < 0.7 else 2
            n_secs = min(n_secs, len(secs))
            secondary_list = random.sample(secs, n_secs)
        else:
            secondary_list = None

        persona = random.choice(personas)

        entry = generate_journal_entry(
            primary_emotion=e,
            secondary_emotions=secondary_list,
            length=length,
        )

        rows3.append({
            "text": entry,
            "primary": e,
            "secondary": ", ".join(secondary_list) if secondary_list else None,
            "length_tag": length,
            "persona_tag": persona,
        })

df3 = pd.DataFrame(rows3)
df3.head(), len(df3)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

(                                                text primary  \
 0  assistant\n\nMy mom's hands smell like the per...     joy   
 1  assistant\n\nI'm still buzzing from last night...     joy   
 2  assistant\n\nI woke up this morning to the sou...     joy   
 3  assistant\n\nI woke up before the sun rose ove...     joy   
 4  assistant\n\nToday was one of those days where...     joy   
 
            secondary length_tag         persona_tag  
 0             relief     medium  parent of two kids  
 1          gratitude      short    unemployed adult  
 2  relief, gratitude     medium      retired person  
 3          gratitude     medium   mid-career worker  
 4               None     medium  parent of two kids  ,
 720)

## Batch 4: tone tags

In [ ]:
# Batch 4 – another diverse batch, with a "tone_tag" for later analysis

tone_tags = ["low_intensity", "mixed_intensity", "high_intensity"]

rows4 = []

for e in emotions:
    for _ in range(80):  # again 80 per primary
        length = random.choice(length_choices)
        secs = secondary_map.get(e, [])
        if secs and random.random() < 0.5:
            secondary = [random.choice(secs)]
        else:
            secondary = None

        persona = random.choice(personas)
        tone_tag = random.choice(tone_tags)

        entry = generate_journal_entry(
            primary_emotion=e,
            secondary_emotions=secondary,
            length=length,
        )

        rows4.append({
            "text": entry,
            "primary": e,
            "secondary": secondary[0] if secondary else None,
            "length_tag": length,
            "persona_tag": persona,
            "tone_tag": tone_tag,
        })

df4 = pd.DataFrame(rows4)
df4.head(), len(df4)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


## Combine batches

In [ ]:
# Make sure all dataframes have the same columns before concat

# Start with list of all dfs
dfs = [df, df2, df3, df4]

# Collect all column names that appear anywhere
all_cols = set()
for d in dfs:
    all_cols.update(d.columns.tolist())

all_cols = list(all_cols)

# Ensure each df has all columns (fill missing with None)
aligned_dfs = []
for idx, d in enumerate(dfs, start=1):
    d = d.copy()
    for c in all_cols:
        if c not in d.columns:
            d[c] = None
    d["batch_id"] = idx  # track origin batch
    aligned_dfs.append(d[all_cols + ["batch_id"]])

df_all = pd.concat(aligned_dfs, ignore_index=True)

print("Total entries:", len(df_all))
df_all.head()


## Save to Drive

In [ ]:
from pathlib import Path
OUTPUT_DIR = Path("/content/drive/MyDrive/VibeQ-EIE/llmdata")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


## Persist metadata

In [ ]:
import json
from datetime import datetime
from pathlib import Path

# Make sure OUTPUT_DIR exists (you already have this somewhere above)
OUTPUT_DIR = Path("/content/drive/MyDrive/VibeQ-EIE/llmdata")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
csv_path = OUTPUT_DIR / f"synthetic_journals_{timestamp}.csv"
jsonl_path = OUTPUT_DIR / f"synthetic_journals_{timestamp}.jsonl"

# Save CSV
df_all.to_csv(csv_path, index=False, encoding="utf-8")

# Save JSONL
with open(jsonl_path, "w", encoding="utf-8") as f:
    for _, row in df_all.iterrows():
        f.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")

csv_path, jsonl_path
